In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

In [2]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
text_df = pd.read_pickle('df_combd_all.pickle')

In [4]:
XY_df = pd.read_pickle('XY_df.pickle')

In [5]:
XY_df = XY_df[['hotel_name','top30','rank_num']]
XY_df.columns

Index([u'hotel_name', u'top30', u'rank_num'], dtype='object')

In [6]:
print text_df.shape, XY_df.shape

(440, 13612) (440, 3)


In [7]:
full_df = pd.concat([text_df,XY_df],axis=1)
full_df.shape

(440, 13615)

In [8]:
hotels_001_100 = full_df[full_df.rank_num <= 100]['content_utf8']

In [9]:
hotels_001_100.shape

(100,)

In [10]:
hotels_001_100.head()

0      we have had numerous stays at hotel one in ny...
1      i stayed 5nights in this hotel and highly rec...
4      we stayed at the 414 for the second time and ...
16     stayed one night they upgraded our room to a ...
18     we stayed for 6 nights during the snow  the r...
Name: content_utf8, dtype: object

In [11]:
hotels_101_200 = full_df[(full_df.rank_num > 100) & (full_df.rank_num <= 200)]['content_utf8']

In [12]:
hotels_201_300 = full_df[(full_df.rank_num > 200) & (full_df.rank_num <= 300)]['content_utf8']

In [13]:
hotels_301_400 = full_df[(full_df.rank_num > 300) & (full_df.rank_num <= 400)]['content_utf8']

In [148]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = list(ENGLISH_STOP_WORDS)
print stop_words

['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'fifty', 'four', 'not', 'own', 'through', 'yourselves', 'go', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neither', 'se

In [149]:
from collections import defaultdict

In [150]:
# remove words that appear freqently
frequency = defaultdict(int)
documents = full_df.content_utf8
for text in documents:
    for token in text.split():
        if token not in frequency:
            frequency[token] = 1
        else:
            frequency[token] += 1

In [151]:
sum(frequency.values()) / float(len(frequency.values()))

93.68193050255613

In [152]:
np.array(frequency.values()).max()

697455

In [153]:
print frequency['hotel'], frequency['staff'], frequency['room'], frequency['nice'], frequency['great'], frequency['comfort'], frequency['love']

142102 64723 121419 33355 63374 1524 5514


In [154]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    token = [word for word in text.split() if frequency[word] < 2500 and word not in stop_words]
    #token = [porter.stem(word) for word in text.split() if frequency[word] < 2500 and word not in stop_words]
    return token

In [35]:
tokenizer_porter("Is it going to rain tomorrow?")

[u'Is', u'go', u'rain', u'tomorrow?']

In [112]:
from nltk import pos_tag

In [107]:
import nltk

In [113]:
pos_tag("My name is Sasha")

[('M', 'NNP'),
 ('y', 'PRP'),
 (' ', 'NNP'),
 ('n', 'VBD'),
 ('a', 'DT'),
 ('m', 'NN'),
 ('e', 'NN'),
 (' ', 'NN'),
 ('i', 'NN'),
 ('s', 'VBP'),
 (' ', 'NNP'),
 ('S', 'NNP'),
 ('a', 'DT'),
 ('s', 'NN'),
 ('h', 'NN'),
 ('a', 'DT')]

In [155]:
# texts_001_100 = [[token for token in text.split() if token not in stop_words]
#           for text in hotels_001_100]
texts_001_100 = [tokenizer_porter(text) for text in hotels_001_100]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts_001_100)

# Create corpus matrix
corpus_001_100 = [dictionary.doc2bow(text) for text in texts_001_100]

In [120]:
# tokens_001_100 = nltk.word_tokenize(texts_001_100)
# #text_001_100 = [word for word in tokens_001_100]
# tagged_001_100 = nltk.pos_tag(tokens_001_100)

In [156]:
lda = models.ldamodel.LdaModel(
    corpus = corpus_001_100,
    id2word=dictionary,
    num_topics = 3,
    passes = 2)

In [157]:
lda.print_topics()

[(0,
  u'0.002*"upgrade" + 0.002*"cheese" + 0.002*"greeted" + 0.002*"chelsea" + 0.002*"library" + 0.002*"delicious" + 0.001*"pool" + 0.001*"ritz" + 0.001*"luxury" + 0.001*"fabulous"'),
 (1,
  u'0.002*"pool" + 0.002*"delicious" + 0.002*"luxury" + 0.002*"roof" + 0.001*"bottle" + 0.001*"certainly" + 0.001*"greeted" + 0.001*"touch" + 0.001*"bryant" + 0.001*"glass"'),
 (2,
  u'0.002*"fabulous" + 0.002*"fun" + 0.002*"continental" + 0.001*"touch" + 0.001*"team" + 0.001*"recommended" + 0.001*"pool" + 0.001*"truly" + 0.001*"decorated" + 0.001*"greeted"')]

In [160]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus_001_100, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      37.775479        1       1  0.009640 -0.004859
2      37.266850        1       2 -0.009967 -0.004396
1      24.957671        1       3  0.000327  0.009255, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
44212  Default   291.000000          414   291.000000  30.0000  30.0000
32893  Default   342.000000       hudson   342.000000  29.0000  29.0000
25416  Default   215.000000     benjamin   215.000000  28.0000  28.0000
43433  Default   432.000000    courtyard   432.000000  27.0000  27.0000
22595  Default   258.000000       broome   258.000000  26.0000  26.0000
30531  Default   252.000000         roxy   252.000000  25.0000  25.0000
37616  Default   290.000000    greenwich   290.000000  24.0000  24.0000
32712  Default   601.000000      chelsea   601.000000  23.0000  23.0000
13595  Default   188.000000      sofitel   188.000000  22.0000  22.0000
27202  Default   248.000000         arlo   248.000000  21.0000  21.0000
42575  Default   517.000000         ritz   517.000000  20.0000  20.0000
11736  Default   257.000000        susan   257.000000  19.0000  19.0000
48747  Default   243.000000        pines   243.000000  18.0000  18.0000
53847  Default   552.000000       bryant   552.000000  17.0000  17.0000
25140  Default   317.000000      kimpton   317.000000  16.0000  16.0000
36829  Default   159.000000       conrad   159.000000  15.0000  15.0000
4857   Default  1031.000000         pool  1031.000000  14.0000  14.0000
19329  Default   188.000000       archer   188.000000  13.0000  13.0000
51319  Default   139.000000      cambria   139.000000  12.0000  12.0000
42554  Default   279.000000        river   279.000000  11.0000  11.0000
45794  Default   179.000000       surrey   179.000000  10.0000  10.0000
53392  Default   213.000000  fitzpatrick   213.000000   9.0000   9.0000
6637   Default   803.000000         roof   803.000000   8.0000   8.0000
5139   Default   208.000000      chatwal   208.000000   7.0000   7.0000
44585  Default   418.000000    microwave   418.000000   6.0000   6.0000
6332   Default   659.000000      library   659.000000   5.0000   5.0000
15103  Default   146.000000     refinery   146.000000   4.0000   4.0000
12260  Default   435.000000        plaza   435.000000   3.0000   3.0000
5323   Default   107.000000        ink48   107.000000   2.0000   2.0000
41618  Default   200.000000          san   200.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
4857    Topic3   333.307615         pool  1031.340645   0.2584  -6.2038
6637    Topic3   268.164089         roof   803.366877   0.2908  -6.4213
21669   Topic3   115.485253     marriott   299.674792   0.4344  -7.2637
48109   Topic3   120.630116          6th   327.517453   0.3892  -7.2201
30069   Topic3   185.465592     checking   572.523335   0.2608  -6.7900
30653   Topic3   270.926214       luxury   922.342704   0.1629  -6.4110
30586   Topic3   279.362615    delicious   970.158910   0.1430  -6.3803
40563   Topic3   212.019723        glass   687.127711   0.2121  -6.6562
31120   Topic3   218.659810    certainly   720.016460   0.1962  -6.6253
51837   Topic3   179.124609       review   560.860533   0.2466  -6.8248
48548   Topic3   227.855053       bottle   778.455224   0.1594  -6.5841
33116   Topic3   181.555867    perfectly   584.521472   0.2188  -6.8113
17637   Topic3   179.278013          spa   574.963479   0.2226  -6.8239
41969   Topic3   206.933526     favorite   716.653598   0.1458  -6.6805
24666   Topic3   205.480739       dining   720.647742   0.1332  -6.6875
5246    Topic3   179.754454  rockefeller   584.140623   0.2094  -6.8213
14282   Topic3   180.803766         spot   608.562309   0.1743  -6.8154
7216    Topic3   186.532065    afternoon   652.455355   0.1358  -6.7843
36340   Topic3 

In [132]:
texts_101_200 = [tokenizer_porter(text) for text in hotels_101_200]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts_101_200)

# Create corpus matrix
corpus_101_200 = [dictionary.doc2bow(text) for text in texts_101_200]

In [133]:
lda = models.ldamodel.LdaModel(
    corpus = corpus_101_200,
    id2word=dictionary,
    num_topics = 3,
    passes = 2)

In [134]:
lda.print_topics()

[(0,
  u'0.003*"request" + 0.003*"marriott" + 0.003*"travel" + 0.002*"hampton" + 0.002*"garden" + 0.002*"return" + 0.002*"point" + 0.002*"surpris" + 0.002*"reserv" + 0.002*"stop"'),
 (1,
  u'0.002*"travel" + 0.002*"price" + 0.002*"reserv" + 0.002*"surpris" + 0.002*"apart" + 0.002*"point" + 0.002*"recommend" + 0.002*"member" + 0.002*"marriott" + 0.002*"live"'),
 (2,
  u'0.003*"travel" + 0.002*"recommend" + 0.002*"live" + 0.002*"request" + 0.002*"point" + 0.002*"stop" + 0.002*"price" + 0.002*"bottl" + 0.002*"store" + 0.002*"serv"')]

In [55]:
texts_201_300 = [tokenizer_porter(text) for text in hotels_201_300]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts_201_300)

# Create corpus matrix
corpus_201_300 = [dictionary.doc2bow(text) for text in texts_201_300]

In [56]:
lda = models.ldamodel.LdaModel(
    corpus = corpus_201_300,
    id2word=dictionary,
    num_topics = 1,
    passes = 2)

In [57]:
lda.print_topics()

[(0,
  u'0.031*"room" + 0.028*"hotel" + 0.017*"stay" + 0.012*"locat" + 0.011*"staff" + 0.011*"great" + 0.010*"time" + 0.008*"clean" + 0.008*"good" + 0.006*"walk"')]

In [161]:
texts_301_400 = [tokenizer_porter(text) for text in hotels_301_400]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts_301_400)

# Create corpus matrix
corpus_301_400 = [dictionary.doc2bow(text) for text in texts_301_400]

In [162]:
lda = models.ldamodel.LdaModel(
    corpus = corpus_301_400,
    id2word=dictionary,
    num_topics = 3,
    passes = 2)

In [163]:
lda.print_topics()

[(0,
  u'0.002*"house" + 0.002*"chelsea" + 0.002*"shared" + 0.001*"budget" + 0.001*"yes" + 0.001*"sink" + 0.001*"dirty" + 0.001*"ac" + 0.001*"pool" + 0.001*"maybe"'),
 (1,
  u'0.002*"chinatown" + 0.002*"bridge" + 0.002*"dirty" + 0.002*"italy" + 0.002*"ac" + 0.001*"toilet" + 0.001*"train" + 0.001*"yes" + 0.001*"budget" + 0.001*"cheap"'),
 (2,
  u'0.002*"toilet" + 0.002*"holiday" + 0.002*"dark" + 0.002*"cheap" + 0.002*"single" + 0.002*"dirty" + 0.001*"sink" + 0.001*"ac" + 0.001*"issues" + 0.001*"shared"')]

In [164]:
pyLDAvis.gensim.prepare(lda, corpus_301_400, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      36.586509        1       1 -0.000620 -0.008057
1      32.344967        1       2 -0.008203  0.004469
2      31.068524        1       3  0.008823  0.003588, topic_info=      Category        Freq         Term       Total  loglift  logprob
term                                                                 
24021  Default  485.000000    chinatown  485.000000  30.0000  30.0000
7794   Default  343.000000       bridge  343.000000  29.0000  29.0000
14616  Default  258.000000       mercer  258.000000  28.0000  28.0000
20555  Default  387.000000        italy  387.000000  27.0000  27.0000
31346  Default  146.000000         jane  146.000000  26.0000  26.0000
3692   Default  299.000000         pool  299.000000  25.0000  25.0000
6924   Default  108.000000       edison  108.000000  24.0000  24.0000
24839  Default  459.000000      chelsea  459.000000  23.0000  23.0000
104    Default  132.000000        hyatt  132.000000  22.0000  22.0000
376    Default  178.000000        union  178.000000  21.0000  21.0000
9640   Default  490.000000      holiday  490.000000  20.0000  20.0000
9824   Default   50.000000    seafarers   50.000000  19.0000  19.0000
36802  Default   80.000000    americano   80.000000  18.0000  18.0000
1981   Default  106.000000      houston  106.000000  17.0000  17.0000
36011  Default   62.000000      redbury   62.000000  16.0000  16.0000
28974  Default   82.000000    paramount   82.000000  15.0000  15.0000
8815   Default   81.000000      mayfair   81.000000  14.0000  14.0000
25308  Default   51.000000        vinci   51.000000  13.0000  13.0000
8684   Default   97.000000         moon   97.000000  12.0000  12.0000
37674  Default   50.000000         ruth   50.000000  11.0000  11.0000
2367   Default   43.000000         elyn   43.000000  10.0000  10.0000
17052  Default   85.000000         mave   85.000000   9.0000   9.0000
37431  Default  492.000000        house  492.000000   8.0000   8.0000
13009  Default   61.000000           da   61.000000   7.0000   7.0000
7548   Default  151.000000         blue  151.000000   6.0000   6.0000
3137   Default  512.000000       shared  512.000000   5.0000   5.0000
38672  Default   81.000000         bunk   81.000000   4.0000   4.0000
27501  Default  472.000000       single  472.000000   3.0000   3.0000
34899  Default  118.000000       circle  118.000000   2.0000   2.0000
9687   Default   66.000000     tenement   66.000000   1.0000   1.0000
...        ...         ...          ...         ...      ...      ...
33143   Topic3  212.216220       toilet  567.944236   0.1846  -6.3716
26526   Topic3  112.320081         shop  269.900099   0.2923  -7.0078
31219   Topic3  143.910389    surprised  363.540998   0.2423  -6.7600
19787   Topic3  169.318411       issues  443.364656   0.2064  -6.5974
3681    Topic3  157.471015         poor  407.548244   0.2181  -6.6699
29520   Topic3   80.249212        awful  184.395479   0.3370  -7.3440
37919   Topic3  178.262148         sink  485.357135   0.1673  -6.5459
13803   Topic3  147.435499         read  391.895100   0.1914  -6.7358
10778   Topic3  191.346199        cheap  547.837662   0.1171  -6.4751
9231    Topic3  126.988206       flight  329.372458   0.2159  -6.8851
22655   Topic3  148.942688         fact  403.721402   0.1718  -6.7256
8392    Topic3  132.672276         turn  350.083378   0.1987  -6.8413
8460    Topic3  164.730335        later  466.291769   0.1285  -6.6249
40558   Topic3  140.348811      waiting  377.736611   0.1789  -6.7850
38644   Topic3  158.576629        phone  447.093138   0.1324  -6.6629
40123   Topic3  166.561828      cleaned  488.200580   0.0936  -6.6138
1198    Topic3  188.623881        dirty  599.328667   0.0129  -6.4894
3137    Topic3  169.246713       shared  512.645773   0.0607  -6.5978
40898   Topic3  154.951728       closet  449.096919   0.1048  -6.6861
26476   Topic3  148.422610           15  424.20

In [14]:
all_text = full_df[['content_utf8','rank_num']]

In [15]:
all_text.shape

(440, 2)

In [16]:
X_text_col = all_text['content_utf8']

In [18]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = list(ENGLISH_STOP_WORDS)

In [19]:
# Preprocess text data to Tfidf
vect = TfidfVectorizer(stop_words=stop_words,ngram_range=(1,2))
X_text = vect.fit_transform(X_text_col)

In [20]:
X_text.shape

(440, 1908825)

In [21]:
X_text_df = pd.DataFrame(X_text.toarray(), columns=vect.vocabulary_)

In [22]:
X_text_df.head()

,cacio room,contrary feel,tv dim,electronic blinds,hunting excellent,cleansheets bit,tv did,residencelocated moma,person add,poor surrounding,...,quiter expected,lady lier,mattress dire,sinkfridge,smelling diptyque,told shed,january january,nights 26th,ounce customer,theme did
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
keep_list = ['upgrade','cheese','delicious','pool','luxury','fabulous','style','touch','atmosphere','fruit','toilet','dark','cheap','dirty','sink','ac','issues','cleaned','closet']

In [25]:
X_text_df = X_text_df[keep_list]

In [26]:
X_text_df.shape

(440, 19)

In [28]:
all_text.columns

Index([u'rank_num'], dtype='object')

In [27]:
X_text_df_w_rank = pd.concat([all_text.drop('content_utf8',axis=1,inplace=True),X_text_df],axis=1)

/Users/sashakapralov/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [29]:
X_text_df_w_rank.shape

(440, 19)

In [30]:
X_text_df_w_rank.columns

Index([u'upgrade', u'cheese', u'delicious', u'pool', u'luxury', u'fabulous',
       u'style', u'touch', u'atmosphere', u'fruit', u'toilet', u'dark',
       u'cheap', u'dirty', u'sink', u'ac', u'issues', u'cleaned', u'closet'],
      dtype='object')

In [31]:
X_text_df_w_rank = pd.concat([all_text,X_text_df],axis=1)

In [32]:
X_text_df_w_rank.shape

(440, 20)

In [33]:
X_text_df_w_rank.to_pickle('X_text_df_w_rank.pickle')